# YOUR PROJECT TITLE

> **Note the following:** 
> 1. This is *not* meant to be an example of an actual **model analysis project**, just an example of how to structure such a project.
> 1. Remember the general advice on structuring and commenting your code
> 1. The `modelproject.py` file includes a function which could be used multiple times in this notebook.

Imports and set magics:

In [178]:
import numpy as np
from scipy import optimize
import sympy as sm

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# local modules
import modelproject

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Model description: The Ramsey-Cass-Koopmans model

Ramsey's model consists of households and firms where households maximize total present-day utility and firms maximize profits. In Ramsey's model, homes live forever, and this is complete foresight. The Ramsey model is often linked to the Solow model, but is simpler in the sense that it does not have to solve as many equations as is customary in Solow. We solve the Ramsey model by performing utility maximization, and for this we need to have the households' utility functions and their income bands, which we use to find the first-order condition. The utility function explains the aggregate utility of the household, while the income curve explains how capital stock changes between periods. We then set up a Lagrange function and end up with two differential equations – one for consumption (the Euler equation) and the other for the capital stock – that describe equilibrium.

The general equilibrium of households and firms with a public sector where consumption is financed by tax revenue, G=T are those c(t) and k(t) that satisfy the following conditions in general:
$$c(t) = \frac{f'(k(t))-\rho-\theta g}{\theta} c(t)$$
$$ k(t)=f(k(t))-c(t)-G(t)-(n+g)k(t) $$
$$ \lim_{t \rightarrow \infty} e^{R(t)}e^{(n+g)t}k_t=0 $$
$$ k(0)=k_0$$

where
$$ R_t = \int\limits_{\tau=0}^t f'(k(\tau)) d\tau $$



We assume that the production function takes on the form of a cobb-douglas
$$Y(t) = K(t)^{\alpha} A(t)L(t)^{1-\alpha}$$
We get the the production per unit:
$$ y(t) =  k(t)^\alpha$$
by defining 
$$k= (K/AL)^\alpha$$
and for the general equilibrium we need to know that: 
$$f'(k(t))= \alpha k(t)^{\alpha -1}$$

The Ramsey-Cass-Koopmans (RCK) model, also known as the neoclassical growth model, is a dynamic optimization model that describes the behavior of an economy over time. It is an extension of the Solow-Swan growth model, incorporating optimizing behavior by households and firms. The RCK model is used to study the optimal allocation of resources between consumption and investment, the dynamics of capital accumulation, and the implications of various economic policies on long-run growth. The model's main components include the utility function, production function, and the consumer's optimization problem.

1. Utility function: The representative household in the RCK model aims to maximize its lifetime utility, which is a function of consumption over time. The utility function is typically assumed to be time-separable, with a constant relative risk aversion (CRRA) form:
$$U(C) = \int \frac{C_t^{1-\sigma} - 1}{1-\sigma} e^{-\rho t} dt$$
2. Production function: The RCK model assumes a neoclassical production function, typically a Cobb-Douglas production function, which exhibits constant returns to scale:
$$Y(t) = K_t^{\alpha} L_t^{1-\alpha}$$
3. Consumer's optimization problem: The representative household in the RCK model seeks to maximize its lifetime utility subject to a budget constraint and the law of motion for capital. The budget constraint states that consumption plus investment must equal output:
$$C_t + I_t = Y_t$$
The law of motion for capital describes how the capital stock evolves over time:
$$ K'_t = I_t - \delta K_t $$

where $K'_t$ is the change in the capital stock, $I_t$ is investment at time t, and $\delta$ is the depreciation rate of capital.

The consumer's optimization problem consists of choosing the optimal consumption and investment paths that maximize lifetime utility, subject to the budget constraint and the law of motion for capital. The solution to this problem yields the Euler equation, which characterizes the optimal consumption and capital accumulation paths in the RCK model.

In [184]:
k_initial = k[0]
y_initial = y[0]
c_initial = c[0]
r_initial = r[0]
w_initial = w[0]

print(c_initial)

1.5215548351171728e-48


**Write out the model in equations here.** 

Make sure you explain well the purpose of the model and comment so that other students who may not have seen it before can follow.  

## Analytical solution

If your model allows for an analytical solution, you should provide here.

You may use Sympy for this. Then you can characterize the solution as a function of a parameter of the model.

To characterize the solution, first derive a steady state equation as a function of a parameter using Sympy.solve and then turn it into a python function by Sympy.lambdify. See the lecture notes for details. 

## Numerical solution

You can always solve a model numerically. 

Define first the set of parameters you need. 

Then choose one of the optimization algorithms that we have gone through in the lectures based on what you think is most fitting for your model.

Are there any problems with convergence? Does the model converge for all starting values? Make a lot of testing to figure these things out. 

# Further analysis

Make detailed vizualizations of how your model changes with parameter values. 

Try to make an extension of the model. 

# Conclusion

Add concise conclusion. 